In [ ]:
import matplotlib as mpl
import matplotlib.patches as mpl_patches
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn

mpl.rc("text", usetex=False)

import cdetools
import corner
import flexcode
import qp
import rail

### Estimating photo-$z$ posterior PDFs with FlexCode

A demonstration of `FlexCode` in the context of photo-$z$s can be found in [Dalmasso, et al 2019](https://arxiv.org/abs/1908.11523), with demos in `R` published on [GitHub](https://github.com/Mr8ND/cdetools_applications).
We'll demonstrate it on the pzflow-based samples generated from the Happy/Teddy data sets.*italicized text*

_This content is adapted from FlexCode's [Teddy tutorial](https://github.com/tpospisi/FlexCode/blob/master/tutorial/Flexcode-tutorial-teddy.ipynb), written by Nic Dalmasso (CMU)._
_The same functionality can also be accessed through a `rail.Estimator()` object; see [tutorial](https://github.com/LSSTDESC/RAIL/blob/master/examples/estimation/RAIL_estimation_demo.ipynb) written by Sam Schmidt (UC Davis)._

In [ ]:
z_min, z_max = 0.0, 1.5
r_min, r_max = 10.0, 25.0
granularity = 100
grid = np.linspace(z_min, z_max, granularity)

n_grid = granularity

# Select regression method
from flexcode.regression_models import NN

# Parameters
basis_system = "cosine"  # Basis system
max_basis = 31  # Maximum number of basis. If the model is not tuned,
# max_basis is set as number of basis

# Regression Parameters
# If a list is passed for any parameter automatic 5-fold CV is used to
# determine the best parameter combination.
params = {
    "k": 5
}  # [5, 10, 15, 20]}       # A dictionary with method-specific regression parameters.

Read Data set and divide into train, validation and test sets

In [ ]:
x_orig = pd.read_csv("../photoz_catalogues/test_set_photometry.csv")[
    ["r", "u-g", "g-r", "r-i", "i-z"]
].to_numpy()
y_orig = pd.read_csv("../photoz_catalogues/test_set_redshifts.csv")[
    ["redshift"]
].to_numpy()
posteriors_orig = pd.DataFrame(
    np.load("../photoz_catalogues/test_set_posteriors.csv")
).to_numpy()

# n_samp = 10000
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test, posteriors_train, posteriors_test = train_test_split(
    x_orig, y_orig, posteriors_orig, train_size=2000, random_state=42
)

(
    x_train,
    x_validation,
    y_train,
    y_validation,
    posteriors_train,
    posteriors_validation,
) = train_test_split(
    x_train, y_train, posteriors_train, train_size=1000, random_state=42
)

In [ ]:
# Parameterize model
model = flexcode.FlexCodeModel(NN, max_basis, basis_system, regression_params=params)

# Fit model - this will also choose the optimal number of neighbors `k`
model.fit(x_train, y_train)

# Tune model - Select the best number of basis
model.tune(x_validation, y_validation)

# Predict new densities on grid
cde_test, y_grid = model.predict(x_test, n_grid=n_grid)

cde_validation, y_grid = model.predict(x_validation, n_grid=n_grid)

We can examine one of the photo-$z$ posteriors estimated with the perfectly representative training/validation set.

In [ ]:
chosen = 9

plt.plot(y_grid, cde_test[chosen], label="Estimated posterior")
plt.plot(grid, posteriors_test[chosen], label="True posterior")
plt.axvline(y_test[chosen], 0, 1, c="C3", label="True redshift")
plt.legend()
plt.xlabel("redshift")
plt.show()

It looks pretty good!
Now let's try training and validating with some Happy/Teddy data but estimating posteriors on the test set from the pzflow demo.

In [ ]:
# y_train = pd.read_csv("teddyAredshifts.csv")["redshift"].to_numpy()
# x_train = pd.read_csv("teddyAphotometry.csv")[
#     ["r", "u-g", "g-r", "r-i", "i-z"]
# ].to_numpy()

# y_validation = pd.read_csv("teddyBredshifts.csv")["redshift"].to_numpy()
# x_validation = pd.read_csv("teddyBphotometry.csv")[
#     ["r", "u-g", "g-r", "r-i", "i-z"]
# ].to_numpy()

In [ ]:
# # Parameterize model
# model = flexcode.FlexCodeModel(NN, max_basis, basis_system, regression_params=params)

# # Fit model - this will also choose the optimal number of neighbors `k`
# model.fit(x_train, y_train)

# # # # Tune model - Select the best number of basis
# model.tune(x_validation, y_validation)

# # # Predict new densities on grid
# cde_test_bias, y_grid_bias = model.predict(x_test, n_grid=n_grid)

In [ ]:
# plt.plot(
#     y_grid_bias,
#     cde_test_bias[chosen],
#     label="Estimated posterior (biased training set)",
# )
# plt.plot(y_grid, cde_test[chosen], label="Estimated posterior (unbiased training set)")
# plt.plot(grid, posteriors_test[chosen], label="True posterior")
# plt.axvline(y_test[chosen], 0, 1, c="C3", label="True redshift")
# plt.legend()
# plt.xlabel("redshift")
# plt.show()

# 2.1 Correcting the PDFs using local regression

In [ ]:
def get_pit(cdes, z_grid, z_test):
    """
    Calculates coverage based upon the CDF

    @param cdes: a numpy array of conditional density estimates;
        each row corresponds to an observation, each column corresponds to a grid
        point
    @param z_grid: a numpy array of the grid points at which cde_estimates is evaluated
    @param z_test: a numpy array of the true z values corresponding to the rows of cde_estimates

    @returns A numpy array of values
    """

    nrow_cde, ncol_cde = cdes.shape
    n_samples = z_test.shape[0]
    n_grid_points = z_grid.shape[0]

    if nrow_cde != n_samples:
        raise ValueError(
            "Number of samples in CDEs should be the same as in z_test."
            "Currently %s and %s." % (nrow_cde, n_samples)
        )
    if ncol_cde != n_grid_points:
        raise ValueError(
            "Number of grid points in CDEs should be the same as in z_grid."
            "Currently %s and %s." % (nrow_cde, n_grid_points)
        )

    z_min = np.min(z_grid)
    z_max = np.max(z_grid)
    z_delta = (z_max - z_min) / (n_grid_points - 1)
    # This line can be vectorized
    vals = [
        z_delta * np.sum(cdes[ii, np.where(z_grid < z_test[ii])[0]])
        for ii in range(n_samples)
    ]
    return np.array(vals)

In [ ]:
pit_values_test = get_pit(
    cde_test, np.squeeze(y_grid, axis=-1), np.squeeze(y_test, axis=-1)
)
pit_values_validation = get_pit(
    cde_validation, np.squeeze(y_grid, axis=-1), np.squeeze(y_validation)
)

## Original Global PIT

In [ ]:
# good model
from plot_utils import plot_with_uniform_band

fig_pit_good = plot_with_uniform_band(
    values=pit_values_test,
    ci_level=0.95,
    x_label="PIT values",
    n_bins=15,
    ylim=[0, 400],
)
fig_pit_good

## Do Global PIT correction

### Theory
The PIT is defined by the transformation 

$$PIT(z) = \int_{0}^{z} f(z'|...)dz'$$

where, $f(z'|...)$ is the conditional density estimate produced by the algorithm. 
If my estimated PDFS are right, then for a given population the true value should lie within the the $q$th quantile for $q$ fraction of times. 

Mathematically, we define the new random variable $P=PIT(z_{true})$. This means the probability distribution for the random variable P, fiven by $F(p)$ should follow a uniform distribution.

In [ ]:
# bins = np.linspace(0,1,101)
num_bins = 100
pops, bins = np.histogram(pit_values_validation, bins=num_bins, density=True)

In [ ]:
correction = np.zeros((len(x_test), n_grid))
for i in range(len(x_test)):
    correction[i] = get_pit(np.tile(cde_test[i],(n_grid,1)), np.squeeze(y_grid), np.squeeze(y_grid) )

In [ ]:
bin_idx = (np.digitize(correction, bins) - 1)
bin_idx[bin_idx==num_bins]=(num_bins-1)

correction = pops[bin_idx]

In [ ]:
cde_test_correct = cde_test*correction
cde_test_correct = cde_test_correct/(np.gradient(np.squeeze(y_grid))*np.sum(cde_test_correct, axis=-1, keepdims=True))

In [ ]:
chosen = 9

plt.plot(y_grid, cde_test_correct[chosen], label="Estimated posterior")
plt.plot(grid, posteriors_test[chosen], label="True posterior")
plt.axvline(y_test[chosen], 0, 1, c="C3", label="True redshift")
plt.legend()
plt.xlabel("redshift")
plt.show()

In [ ]:
pit_values_test_correct = get_pit(
    cde_test_correct, np.squeeze(y_grid, axis=-1), np.squeeze(y_test, axis=-1)
)

In [ ]:
fig_pit_good = plot_with_uniform_band(
    values=pit_values_test_correct,
    ci_level=0.95,
    x_label="PIT values",
    n_bins=15,
    ylim=[0, 400],
)
fig_pit_good

# show that global callibration does not mean local

In [ ]:
corner.corner(x_test)

In [ ]:
from cde_diagnostics.local_histogram import local_histogram

fig = local_histogram(x_train=x_test, pit_train=pit_values_test,
                      x_test=[(0,0,0,0,0)],
                      alphas=np.linspace(0.0, 0.99, 99), clf_name='XGBoost \n (d3, n1000)',
                      ci_level=0.05, n_bins=7, figsize=(5,4)
                     )

In [ ]:
fig.figure

# Next steps:
- for each data point in the test set predict the local PIT based on the dev set, have to do it fast (XGB+GPU?)
- following that use the same procedure as above to find the correction term.
- because of the binned nature of the correction the PDFs become spikey.

## 3. Evaluating the performance of estimated photo-$z$ posterior PDFs

Once we have estimated photo-$z$ posterior PDFs, we need a way to determine if they're actually any good.
Since the tutorial has only one method but multiple training/validation sets, that's all we can compare for now.

### Metrics of estimated photo-$z$ posteriors and true redshifts

First, let's try out a couple metrics of estimated photo-$z$ posteriors that do not require knowledge of the true photo-$z$ posteriors.
There's additional functionality for the case of having true redshifts but not true posteriors in [cdetools](https://github.com/tpospisi/cdetools) and [cde-diagnostics](https://github.com/zhao-david/CDE-diagnostics), but this should give a general idea.

In [ ]:
from cdetools import cde_loss, cdf_coverage, hpd_coverage

The Probability Integral Transform (PIT) is defined as 
\begin{equation}
PIT = \int_{-\infty}^{z_{true}} p(z | \vec{d}, \{z_{n}, \vec{d}_{n}\}_{N}, \pi) dz .
\end{equation}
A histogram of PIT values is commonly used to assess how consistent a population of photo-$z$ PDFs are with the true redshifts.
Ideally, it would be a uniform distribution, meaning N% of galaxies have their true redshift within the Nth percentile of their estimated photo-$z$ posterior PDF.

In [ ]:
pit_values = cdf_coverage.cdf_coverage(cde_test, y_grid, y_test)
pit_values_bias = cdf_coverage.cdf_coverage(cde_test_bias, y_grid_bias, y_test)

plt.hist(pit_values, alpha=0.5, bins=100, label="representative")
plt.hist(pit_values_bias, alpha=0.5, bins=100, label="biased")
plt.ylim(0, 100)
plt.legend()
plt.xlabel("PIT")

The Highest Predictive Density (HPD) 
\begin{equation}
HPD = \int_{z': p(z' | \vec{d}_{i}, \{z_{n}, \vec{d}_{n}\}_{N}, \pi) \geq p(z | \vec{d}_{i}, \{z_{n}, \vec{d}_{n}\}_{N}, \pi)} p(z' | \vec{d}_{i}, \{z_{n}, \vec{d}_{n}\}_{N}, \pi) dz
\end{equation}
is like the area of the PDF where it exceeds a given value.
Over a population, it would ideally be flat, like the PIT.
[A talk by David Zhao (CMU)](https://drive.google.com/file/d/1uvPtK_RcTUHEwt0ZYld41VKEPHnehWbN/view) has a lovely visualization of the HPD.

In [ ]:
hpd_cov = hpd_coverage.hpd_coverage(cde_test, y_grid, y_test)
hpd_cov_bias = hpd_coverage.hpd_coverage(cde_test_bias, y_grid_bias, y_test)

plt.hist(hpd_cov, alpha=0.5, bins=100, label="representative")
plt.hist(hpd_cov_bias, alpha=0.5, bins=100, label="biased")
plt.ylim(0, 100)
plt.legend()
plt.xlabel("HPD")

The CDE loss 
\begin{equation}
\hat{L} = \frac{1}{K} \sum_{i=1}^{K} \int \left(p(z | \vec{d}_{i}, \{z_{n}, \vec{d}_{n}\}_{N}, \pi)\right)^{2} dz - \frac{2}{K} \sum_{i=1}^{K} p(z_{i} | \vec{d}_{i}, \{z_{n}, \vec{d}_{n}\}_{N}, \pi)
\end{equation}
approximates the true posterior from the estimated posterior evaluated at the true redshift.
It's explained quite well in [a talk by Nic Dalmasso (CMU)](https://www.dropbox.com/s/2r4tl4qv0iyqo9b/STAMPS_LSST_CDE_Tools_Presentation.pdf?dl=0).
A lower value indicates a better estimator.

In [ ]:
print(cde_loss.cde_loss(cde_test, y_grid, y_test))
print(cde_loss.cde_loss(cde_test_bias, y_grid_bias, y_test))

### Comparison of estimated and true photo-$z$ posterior PDFs

There are two categories of metrics of approximated and true PDF: 
those that either rely upon or force the normalization condition $\int p(z) dz = 1$and those that evaluate differences between arbitrary functions.
`qp` [(Malz, et al 2018)](https://arxiv.org/abs/1806.00014) is a package for manipulating univariate PDFs under many parameterizations and includes a few comparison metrics.

The [original version](https://github.com/aimalz/qp) consistently enforced normalization but had limited functionality, whereas the [new version](https://github.com/LSSTDESC/qp) includes many more parameterizations whose usage is "at your own risk" in terms of possibly violating normalization.
We'll use the new version for the sake of speed but evaluating metrics using simplified functions ported from the old version due to a (hopefully transient) bug in the handling of large sets of posteriors.
The first step is to get both the true posteriors and the approximations evaluated on the same grid of redshifts.

_This content is adapted from the [qp demo](https://github.com/LSSTDESC/qp/blob/master/docs/notebooks/demo.ipynb), written by Alex Malz (GCCL@RUB), Phil Marshall (SLAC), and Eric Charles (SLAC), and [qp metrics demo](https://github.com/LSSTDESC/qp/blob/master/docs/notebooks/kld.ipynb), written by Alex Malz (GCCL@RUB) and Phil Marshall (SLAC)._

In [ ]:
# P = qp.Ensemble(qp.interp, data=dict(xvals=grid.reshape(grid.shape[0]), yvals=posteriors_test))
Q = qp.Ensemble(
    qp.interp, data=dict(xvals=y_grid.reshape(y_grid.shape[0]), yvals=cde_test)
)
Q_bias = qp.Ensemble(
    qp.interp,
    data=dict(xvals=y_grid.reshape(y_grid_bias.shape[0]), yvals=cde_test_bias),
)
grid, approx_pdf_on_grid = Q.gridded(grid)
grid, approx_pdf_on_grid_bias = Q_bias.gridded(grid)

The Kullback Leibler Divergence (KLD)
\begin{equation}
KLD = \int_{-\infty}^{\infty} p(z | \vec{d}) \log\left[\frac{p(z | \vec{d}, \{z_{n}, \vec{d}_{n}\}_{N}, \pi)}{p(z | \vec{d})}\right] dz
\end{equation}
is a directional measure of how much information is lost by using the estimated $p(z | \vec{d}, \{z_{n}, \vec{d}_{n}\}_{N}, \pi)$ instead of the true $p(z | \vec{d})$.
We want the KLD for each galaxy to be low.

In [ ]:
KLDs = np.array(
    [
        qp.metrics.quick_kld(p, q, dx=np.mean(grid[1:] - grid[:-1]))
        for p, q in zip(posteriors_test, approx_pdf_on_grid)
    ]
)
KLDs_bias = np.array(
    [
        qp.metrics.quick_kld(p, q, dx=np.mean(grid[1:] - grid[:-1]))
        for p, q in zip(posteriors_test, approx_pdf_on_grid_bias)
    ]
)

plt.hist(np.log(KLDs), alpha=0.5, bins=100, label="representative", density=True)
plt.hist(np.log(KLDs_bias), alpha=0.5, bins=100, label="biased", density=True)
plt.xlabel("KLD")
plt.legend()
plt.semilogy()

The root-mean-square-error (RMSE) is a symmetric measure commonly used to compare 1D functions.
**TODO: write it out?** Similarly, a lower value corresponds to a more closely approximating posterior PDF.

In [ ]:
RMSEs = np.array(
    [
        qp.metrics.quick_rmse(p, q, N=granularity)
        for p, q in zip(posteriors_test, approx_pdf_on_grid)
    ]
)
RMSEs_bias = np.array(
    [
        qp.metrics.quick_rmse(p, q, N=granularity)
        for p, q in zip(posteriors_test, approx_pdf_on_grid_bias)
    ]
)

plt.hist(np.log(RMSEs), alpha=0.5, bins=100, label="representative", density=True)
plt.hist(np.log(RMSEs_bias), alpha=0.5, bins=100, label="biased", density=True)
plt.xlabel("RMSE")
plt.legend()

### Your turn!

How else can we quantify the performance of estimators of aleatoric uncertainty?

**CHALLENGE**: Apply and visualize the local metrics of [Zhao, Dalmasso, Izbicki & Lee, 2021](https://arxiv.org/abs/2102.10473), or any other metrics not included in this demo; the [cde-diagnostics tutorial](https://github.com/zhao-david/CDE-diagnostics/blob/main/tutorial/tutorial-cde-diagnostics.ipynb), written by David Zhao (CMU), may be a good starting point.

**CHALLENGE**: Use the differences between estimated and true photo-$z$ posterior PDFs as a function of photometry to isolate the implicit prior of epistemic uncertainty from the aleatoric uncertainty.

# Your turn: How to participate in this challenge tl;dr

The organizers have identified three hack-able aspects to this data challenge:
- Quantify sensitivity of ML photo-z posterior estimators to training set non-representativity
- Implement and test additional AI methods for photo-z posterior estimation
- Implement/interpret additional metrics of posterior precision

Of course, there are many other possibilities for what to investigate based on this starting material!

To participate, clone this tutorial's [GitHub repo](https://github.com/aimalz/qtc2021), make or comment on [an issue](https://github.com/aimalz/qtc2021/issues) saying what you'd like to hack on, and work in a branch corresponding to that issue.
The only rule is that if your hack leads to a publication, you cite the repo (and, of course, invite those who contributed to your solution to be authors).